In [ ]:
"""Import modules"""

# pip freeze | grep -E "pandas|numpy|matplotlib|seaborn|math|pickle|random|statistics|datetime|folium|scipy|scikit-posthocs|statsmodels|sklearn|tensorflow" > requirements.txt

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import pickle
import random
import statistics
from datetime import datetime
import folium
from scipy import stats
import scikit_posthocs as sp
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf
import time

from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.decomposition import PCA
from sklearn.impute import KNNImputer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import silhouette_score
from sklearn.model_selection import TimeSeriesSplit

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("TensorFlow version:", tf.__version__)

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Layer
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import SGD

from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Dropout, Bidirectional, LSTM, Input
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import custom_object_scope

# pip cache purge
!pip show tensorflow


Num GPUs Available:  1
TensorFlow version: 2.10.0
Name: tensorflow
Version: 2.10.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /home/huyushin/miniconda3/envs/aibld/lib/python3.8/site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, keras-preprocessing, libclang, numpy, opt-einsum, packaging, protobuf, setuptools, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: 


In [ ]:
"""Read data"""

path = 'data/Available building EUI (2014-2021).csv'
df_original = pd.read_csv(path, parse_dates=True)
df_original = df_original.rename(columns=lambda x: int(x) if x.isdigit() else x)


In [ ]:
"""Building information"""

available_buildingli = [2, 4, 5, 7, 8, 9, 11, 14, 16, 20, 21, 23, 24, 25, 26, 28, 31, 32, 34, 35, 37, 39, 40, 42, 45, 48, 49, 50, 52, 56, 61, 63, 65, 67, 68, 69, 70, 71, 74, 75, 77, 78, 80, 82, 83, 88, 90, 92, 94, 95, 97, 99, 101, 108, 109, 110, 111, 114, 115, 118, 119, 120, 121, 123, 125, 128, 129, 130, 132, 133, 135, 140, 142, 146, 147, 153, 156, 158, 163, 164, 166, 167, 168, 170, 171, 181, 182, 184, 185]
building_classdict = {2: 'Research building', 4: 'Mix-used building', 5: 'Research building', 7: 'Mix-used building', 8: 'Research building', 9: 'Others', 11: 'Research building', 14: 'Others', 16: 'Research building', 20: 'Research building', 21: 'Research building', 23: 'Research building', 24: 'Research building', 25: 'Research building', 26: 'Research building', 28: 'Research building', 31: 'Research building', 32: 'Research building', 34: 'Research building', 35: 'Research building', 37: 'Research building', 39: 'Library', 40: 'Research building', 42: 'Research building', 45: 'Mix-used building', 48: 'Office building', 49: 'Research building', 50: 'Research building', 52: 'Research building', 56: 'Research building', 61: 'Research building', 63: 'Research building', 65: 'Research building', 67: 'Research building', 68: 'Research building', 69: 'Mix-used building', 70: 'Research building', 71: 'Mix-used building', 74: 'Research building', 75: 'Research building', 77: 'Others', 78: 'Dormitory', 80: 'Others', 82: 'Teaching building', 83: 'Research building', 88: 'Research building', 90: 'Dormitory', 92: 'Dormitory', 94: 'Research building', 95: 'Research building', 97: 'Others', 99: 'Dormitory', 101: 'Dormitory', 108: 'Research building', 109: 'Dormitory', 110: 'Mix-used building', 111: 'Office building', 114: 'Sports center', 115: 'Dormitory', 118: 'Teaching building', 119: 'Research building', 120: 'Research building', 121: 'Dormitory', 123: 'Teaching building', 125: 'Others', 128: 'Dormitory', 129: 'Others', 130: 'Teaching building', 132: 'Research building', 133: 'Others', 135: 'Research building', 140: 'Research building', 142: 'Research building', 146: 'Research building', 147: 'Teaching building', 153: 'Others', 156: 'Others', 158: 'Research building', 163: 'Dormitory', 164: 'Research building', 166: 'Research building', 167: 'Research building', 168: 'Research building', 170: 'Research building', 171: 'Teaching building', 181: 'Mix-used building', 182: 'Mix-used building', 184: 'Research building', 185: 'Office building'}
building_namedict = {2: '計算機及資訊網路中心', 4: '普通學生餐廳(福利社)', 5: '原分所', 7: '男一舍餐廳', 8: '電機一館[舊電機館(原稱電機工程研究所)]', 9: '辛亥路大門.警衛室', 11: '電機二館', 14: '大門口警衛室(古蹟)', 16: '農化系實驗室', 20: '物理學系凝態科學研究中心', 21: '地球科學大樓(全球變遷中心)', 23: '林產館', 24: '化學工程館', 25: '一號館', 26: '植病系昆蟲館、植病系昆蟲館養蟲室8、昆蟲館養蟲室6(植病系昆蟲館養蟲室6)', 28: '園藝花卉研究室', 31: '電子顯微鏡館', 32: '獸醫館(獸醫系研究室)', 34: '生命科學館', 35: '展書樓(編譯館(舊))、展書樓(國立編譯館新館)', 37: '食品科技館', 39: '總圖書館(總圖新館)', 40: '園產加工實驗室', 42: '農藝館', 45: '學生活動中心', 48: '農化系食品工廠(收發室)', 49: '生技中心', 50: '博理館', 52: '生物化學研究所、生化所擴建', 56: '畜產系畜牧大樓(動物科學技術學系)、畜產加工館', 61: '四號館(園藝系館)、園藝玻璃溫室', 63: '獸醫系三館', 65: '工學院綜合大樓', 67: '應用力學研究大樓', 68: '納環館 (環工所)', 69: '數學館', 70: '農業化學系研究室', 71: '雅頌坊', 74: '家畜醫院新大樓(動物醫院)', 75: '食品研發大樓', 77: '動物標本館', 78: '男七舍', 80: '司令臺', 82: '外語教學暨資源中心', 83: '數學研究中心', 88: '航空測量館', 90: '第二男女研究生宿舍(大一女舍)', 92: '女八舍、女九舍', 94: '森林館', 95: '五號館(農工系館)', 97: '事務組倉庫、司機室(原電話總機室)', 99: '男八舍', 101: '國青大樓', 108: '地理系館', 109: '男三舍', 110: '學生保健中心', 111: '望樂樓(植研大樓)', 114: '體育館(校總區)', 115: '男五舍', 118: '共同教學館', 119: '水工試驗大樓', 120: '思亮館', 121: '女一舍', 123: '新生教學館', 125: '普通物理系倉庫', 128: '女五舍', 129: '水產養殖池', 130: '文學院(古蹟)', 132: '土木館', 133: '植物標本館', 135: '管理學院一號館(管理學院)', 140: '萬才館', 142: '管理學院二號館', 146: '霖澤館', 147: '園藝系造園館', 153: '倉庫(保管組及會計室)', 156: '農業陳列館', 158: '農化新館/敬賢樓(農化二館暨行政大樓)', 163: '第一女研究生宿舍', 164: '國家發展所大樓(原三研所)', 166: '社會及社工館(社會系所大樓)', 167: '新聞所大樓', 168: '種子研究室', 170: '農工系實驗室(生物環境系統工程學系)', 171: '建城所(原內政部借用房屋)(檔案樓)', 181: '小小福', 182: '農產品展售中心', 184: '心理系南館', 185: '農業試驗場辦公室'}
building_areadict = {2: 4886.19, 4: 1728.91, 5: 9321.7, 7: 847.62, 8: 2139.55, 9: 22.64, 11: 16007.3, 14: 44.36, 16: 890.58, 20: 26292.17, 21: 2629.69, 23: 1129.95, 24: 4653.18, 25: 4060.56, 26: 1313.7800000000002, 28: 1381.32, 31: 500.17, 32: 3685.94, 34: 28817.6, 35: 3899.9300000000003, 37: 3018.73, 39: 35325.15, 40: 1519.17, 42: 3989.33, 45: 5849.3, 48: 585.64, 49: 8512.63, 50: 11460.99, 52: 5112.610000000001, 56: 9271.59, 61: 3675.62, 63: 2994.29, 65: 24629.66, 67: 11571.39, 68: 3732.32, 69: 1431.92, 70: 706.47, 71: 426.31, 74: 10478.93, 75: 1998.98, 77: 199.78, 78: 6340.37, 80: 144.89, 82: 4454.37, 83: 2693.46, 88: 565.64, 90: 10052.41, 92: 3766.75, 94: 3766.26, 95: 2288.55, 97: 316.48, 99: 6769.99, 101: 13034.04, 108: 4005.47, 109: 6110.91, 110: 1159.21, 111: 1110.81, 114: 41448.4, 115: 4090.12, 118: 5460.55, 119: 3809.2, 120: 10199.82, 121: 5297.85, 123: 5201.01, 125: 46.24, 128: 1771.83, 129: 592.12, 130: 6484.64, 132: 5922.72, 133: 1095.95, 135: 12968.47, 140: 9261.39, 142: 10816.4, 146: 9800.35, 147: 1490.27, 153: 412.66, 156: 1800.0, 158: 9107.02, 163: 4778.49, 164: 6428.71, 166: 7902.38, 167: 6231.4, 168: 383.05, 170: 804.81, 171: 1684.55, 181: 160.0, 182: 66.107, 184: 3595.312, 185: 504.714}
building_yeardict = {2: 1982, 4: 1981, 5: 1985, 7: 1980, 8: 1969, 9: np.nan, 11: 1993, 14: np.nan, 16: 1923, 20: 2000, 21: 1974, 23: 1987, 24: 1962, 25: 1930, 26: 1936, 28: 1990, 31: np.nan, 32: 1987, 34: 1999, 35: 1956, 37: 1983, 39: 1997, 40: 1976, 42: 1976, 45: 1962, 48: 1929, 49: 1987, 50: 2004, 52: 1974, 56: 1961, 61: 1930, 63: 2005, 65: 1992, 67: 1987, 68: 2015, 69: 1962, 70: np.nan, 71: 2009, 74: 1995, 75: 2003, 77: np.nan, 78: 1987, 80: np.nan, 82: 1982, 83: 1970, 88: 1960, 90: 1992, 92: 1958, 94: 1957, 95: 1935, 97: np.nan, 99: 1993, 101: 1978, 108: 1979, 109: 1983, 110: 1965, 111: 1968, 114: 2001, 115: 1989, 118: 1984, 119: 1989, 120: 1983, 121: 1977, 123: 1961, 125: np.nan, 128: 1956, 129: 1984, 130: 1929, 132: 1955, 133: 1929, 135: 1994, 140: 2009, 142: 2001, 146: 2009, 147: 1987, 153: 1966, 156: 1961, 158: 1994, 163: 1988, 164: np.nan, 166: np.nan, 167: 1996, 168: np.nan, 170: np.nan, 171: 1981, 181: 1956, 182: 1932, 184: 1990, 185: 1925}
building_floordict = {2: 4, 4: 3, 5: 4, 7: 1, 8: 4, 9: np.nan, 11: 5, 14: np.nan, 16: 1, 20: 14, 21: 4, 23: 4, 24: 3, 25: 3, 26: 2, 28: 4, 31: np.nan, 32: 4, 34: 12, 35: 3, 37: 4, 39: 5, 40: 2, 42: 4, 45: 2, 48: 1, 49: 6, 50: 7, 52: 4, 56: 3, 61: 2, 63: 5, 65: 7, 67: 4, 68: 4, 69: 3, 70: np.nan, 71: 1, 74: 9, 75: 4, 77: np.nan, 78: 4, 80: np.nan, 82: 3, 83: 4, 88: 2, 90: 5, 92: 3, 94: 4, 95: 2, 97: np.nan, 99: 7, 101: 11, 108: 8, 109: 4, 110: 2, 111: 3, 114: 6, 115: 4, 118: 4, 119: 4, 120: 6, 121: 4, 123: 5, 125: np.nan, 128: 3, 129: 1, 130: 2, 132: 4, 133: 2, 135: 10, 140: 7, 142: 11, 146: 8, 147: 3, 153: 1, 156: 3, 158: 5, 163: 7, 164: np.nan, 166: np.nan, 167: 4, 168: np.nan, 170: np.nan, 171: 4, 181: 1, 182: 1, 184: 4, 185: 1}
building_undergroungfloordict = {2: 1, 4: 1, 5: 1, 7: 0, 8: 1, 9: np.nan, 11: 1, 14: np.nan, 16: 0, 20: 1, 21: 1, 23: 1, 24: 1, 25: 0, 26: 0, 28: 1, 31: np.nan, 32: 1, 34: 1, 35: 1, 37: 1, 39: 1, 40: 1, 42: 1, 45: 1, 48: 0, 49: 2, 50: 1, 52: 1, 56: 0, 61: 0, 63: 1, 65: 1, 67: 1, 68: 1, 69: 0, 70: np.nan, 71: 0, 74: 1, 75: 1, 77: np.nan, 78: 1, 80: np.nan, 82: 1, 83: 1, 88: 0, 90: 1, 92: 0, 94: 0, 95: 0, 97: np.nan, 99: 1, 101: 1, 108: 1, 109: 1, 110: 0, 111: 0, 114: 2, 115: 1, 118: 0, 119: 1, 120: 1, 121: 1, 123: 0, 125: np.nan, 128: 0, 129: 0, 130: 0, 132: 0, 133: 0, 135: 1, 140: 1, 142: 0, 146: 1, 147: 0, 153: 0, 156: 0, 158: 1, 163: 1, 164: np.nan, 166: np.nan, 167: 1, 168: np.nan, 170: np.nan, 171: 1, 181: 0, 182: 0, 184: 1, 185: 0}
class_buildingdict = {'Research building': [2, 5, 8, 11, 16, 20, 21, 23, 24, 25, 26, 28, 31, 32, 34, 35, 37, 40, 42, 49, 50, 52, 56, 61, 63, 65, 67, 68, 70, 74, 75, 83, 88, 94, 95, 108, 119, 120, 132, 135, 140, 142, 146, 158, 164, 166, 167, 168, 170, 184], 'Mix-used building': [4, 7, 45, 69, 71, 110, 181, 182], 'Others': [9, 14, 77, 80, 97, 125, 129, 133, 153, 156], 'Library': [39], 'Office building': [48, 111, 185], 'Dormitory': [78, 90, 92, 99, 101, 109, 115, 121, 128, 163], 'Teaching building': [82, 118, 123, 130, 147, 171], 'Sports center': [114]}
buildingtypeli = ['Teaching building', 'Research building', 'Office building', 'Mix-used building', 'Library', 'Sports center', 'Dormitory', 'Others']
